In [14]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [0]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score, KFold
import eli5
import xgboost as xgb
from eli5.sklearn import PermutationImportance
from hyperopt import hp,fmin,tpe,STATUS_OK

In [16]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [17]:
df=pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Feature Engineering

In [0]:
SUFFIX_CAT='__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list): continue
  factorized_values=df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat]=factorized_values
  else:
    df[feat+SUFFIX_CAT]=factorized_values

In [0]:
cat_feats=[x for x in df.columns if SUFFIX_CAT in x]
cat_feats=[x for x in cat_feats if 'price' not in x]

In [0]:
df['param_rok-produkcji']=df['param_rok-produkcji'].map(lambda x: -1 if str(x)=='None' else int(x))
df['param_moc']=df['param_moc'].map(lambda x: -1 if x==None else int(x.split(' ')[0]))
df['param_pojemność-skokowa']=df['param_pojemność-skokowa'].map(lambda x: -1 if x==None else int(x.split('cm')[0].replace(' ','')))

In [0]:
def run_model(model,feats):
  X=df[feats].values
  y=df['price_value'].values
  scores=cross_val_score(model,X,y,cv=3,scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
feats=['param_napęd__cat',
'param_rok-produkcji',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'param_pojemność-skokowa',
'seller_name__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'param_wersja__cat',
'param_kod-silnika__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_regulowane-zawieszenie__cat']

In [0]:
xgb_params={'max_depth':5,'n_estimators':50,'learning_rate':.1,'seed':0}

In [24]:
run_model(xgb.XGBRegressor(**xgb_params),feats)

[15:39:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:39:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:39:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)

In [0]:
def obj_func(params):
  print("Training with params: ")
  print(params)

  mean_mae,score_std=run_model(xgb.XGBRegressor(**params),feats)
  return {'loss':np.abs(mean_mae),'status':STATUS_OK}

In [0]:
xgb_reg_params={
    'learning_rate':hp.choice('learning_rate',np.arange(0.05,0.31,0.05)),
    'max_depth':hp.choice('max_depth',np.arange(5,16,1,dtype=int)),
    'subsample':hp.quniform('subsample',0.5,1,0.05),
    'colsample_bytree':hp.quniform('colsample_bytree',0.5,1,.05),
    'objective':'reg:squarederror',
    'n_estimators':100,
    'seed':0
}

In [28]:
best=fmin(obj_func,xgb_reg_params,algo=tpe.suggest,max_evals=25)
best

Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.15000000000000002, 'max_depth': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
Training with params: 
{'colsample_bytree': 0.9, 'learning_rate': 0.3, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 0.55, 'learning_rate': 0.25, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 0.9, 'learning_rate': 0.2, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
Training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 

{'colsample_bytree': 0.8, 'learning_rate': 0, 'max_depth': 8, 'subsample': 0.8}